In [1]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
import torch
import pandas as pd
from tqdm import tqdm
import gc
import re
import time
from typing import Dict, Any
import logging
import sys
import json

class SinglePassProcessor:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.model.eval()
        
    def process_note(self, person_id, note_date, note_text, strong_evidence_example, weak_evidence_example, phenotype):
        """Process a note in a single model pass""" 
        print("Creating prompt...")
        with torch.no_grad():
                prompt = f"""[INST]
                Task: You are a clinical expert analyzing patient notes for evidence of recent suicide attempt. Your main task is to carefully read the notes and classify them as either indicating a suicide attempt or not indicating a suicide attempt. 

                CLASSIFICATION must be a single number:
                   0: Does not indicate suicide attempt
                   1: Indicates suicide attempt
                
                Example clinical notes, analyses, and responses:
                {weak_evidence_example}
                {strong_evidence_example}
                
                Analyze the following clinical note: "{note_text}"

                IMPORTANT: YOU MUST FORMAT YOUR RESPONSE INTO A JSON FORMAT EXACTLY AS FOLLOWS:
                {{
                "person_id": {person_id}, 
                "note_date": "{note_date}", 
                "CLASSIFICATION": <Single number only: must be 0 or 1>
                }}
                IMPORTANT: ONLY OUTPUT THE JSON FORMAT RESPONSE, NOTHING ELSE.
                [/INST]
                """
                
                messages = [{"role": "user", "content": prompt}]
                
                inputs = self.tokenizer.apply_chat_template(
                                messages,
                                tokenize=True,
                                add_generation_prompt=True,
                                return_tensors="pt",
                            ).to("cuda")
                input_length = inputs.size(1)
                
                # Generate model outputs
                outputs = self.model.generate(
                    input_ids=inputs,
                    max_new_tokens=100,
                    use_cache=True,
                    temperature=0.1
                )
    
                generated_tokens = outputs[:, input_length:]
                # Decode and store the result
                response = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
                
                return response

def main():
    def append_to_json_file(response_str, file_path):
        try:
            # Find the JSON content using regex
            json_pattern = r'\{.*\}'
            json_match = re.search(json_pattern, response_str, re.DOTALL)
            print(response_str)
            if json_match:
                json_content = json_match.group(0).strip()
                
                # Load existing array if file exists, or create new array
                try:
                    with open(file_path, 'r') as file:
                        results = json.load(file)
                except (FileNotFoundError, json.JSONDecodeError):
                    results = []
                
                # Append new content
                results.append(json.loads(json_content))
                
                # Write back the updated array
                with open(file_path, 'w') as file:
                    json.dump(results, file, indent=2)
            else:
                print("No valid JSON content found in response")
        except Exception as e:
            print(f"Error writing to file: {str(e)}")
            
    try:
        max_seq_length = 2048
        dtype = None
        load_in_4bit = False
        
        finetuned_model, tokenizer = FastLanguageModel.from_pretrained(
            # model_name = "../Llama3_2_1B/",
            model_name = "../Llama3_2_3B/",
            # model_name = "../Llama3_2_3B_FineTuned/llama-3.2-lora_model",
            max_seq_length = max_seq_length,
            dtype = dtype,
            load_in_4bit = load_in_4bit
        )
        
        tokenizer = get_chat_template(
            tokenizer,
            chat_template = "llama-3.1",
        )
        
        FastLanguageModel.for_inference(finetuned_model)
        
        phenotype = "suicide_attempt"
        with open(f"../data/{phenotype}/weak_evidence_short.txt", "r") as f:
            weak_evidence_example = f.read()
        with open(f"../data/{phenotype}/strong_evidence_short.txt", "r") as f:
            strong_evidence_example = f.read()

        df = pd.read_csv(f"../data/{phenotype}/FDA_Chart_Cleaned.csv")
        
        # Filepath to the text file
        # output_file_path = f"../data/{phenotype}/Llama3_2_1B_Instruct_3_SA_3_SI_context_SA_output.json"
        output_file_path = f"../data/{phenotype}/Llama3_2_3B_Instruct_3_SA_3_SI_context_SA_output.json"
        # output_file_path = f"../data/{phenotype}/Llama3_2_3B_Finetuned_3_SA_3_SI_context_SA_output.json"

        print("Initializing processor...")
        processor = SinglePassProcessor(finetuned_model, tokenizer)
        print("Processor initialized")

        # Process notes with progress bar
        for index in tqdm(range(len(df)), desc="Processing notes"):          
            try: 
                if index == 139:
                    continue  

                print(f"\nProcessing note {index + 1}/{len(df)}")
                person_id = df.iloc[index]["person_id"]
                note_date = df.iloc[index]["note_date"]
                note_text = df.iloc[index]["note_text"]
                
                # Process note in a single pass with timing
                start_time = time.time()
                response = processor.process_note(
                    person_id,
                    note_date,
                    note_text,
                    strong_evidence_example,
                    weak_evidence_example,
                    phenotype
                )
                append_to_json_file(response, output_file_path)

                end_time = time.time()
                processing_time = end_time - start_time
                print(f"Note processed in {processing_time:.2f} seconds")
                
                # Memory cleanup
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    print("Memory cleaned")
                    
            except Exception as e:
                print(f"Error processing note at index {index}: {str(e)}")
                import traceback
                traceback.print_exc()
                continue

        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("Memory cleaned")
        print("Processing completed successfully")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

/home/leeh65@ds.vumc.io/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.431 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████████████| 2/2 [00:02<00:00,  1.33s/it]
../Llama3_2_3B/ does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


Initializing processor...
Processor initialized


Processing notes:   0%|                                 | 0/363 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


...

Processing notes: 100%|██████████████████████▉| 362/363 [22:05<00:09,  9.11s/it]

                "person_id": 1900
                "note_date": "2020-12-28", 
                "note_date": "2020-12-28", 
                "note_date": "2020-12-28", 
                "note_date": "2020-12-28", 
                "note_date": "2020-12-28", 
                "note_date": "2020-12-28", 
                "note_date": "202
No valid JSON content found in response
Note processed in 22.33 seconds
Memory cleaned

Processing note 363/363
Creating prompt...
{
    "person_id": 190554294, 
    "note_date": "2020-12-09", 
    "CLASSIFICATION": 1
}
Note processed in 2.43 seconds


Processing notes: 100%|███████████████████████| 363/363 [22:07<00:00,  3.66s/it]

Memory cleaned
Memory cleaned
Processing completed successfully



W1205 18:52:17.178000 140073902388992 torch/_inductor/compile_worker/subproc_pool.py:126] SubprocPool unclean exit
